# Regulizadores

Neste código iremos analisar como funcionam os regularizadores e como eles são usados para evitar *overfitting*, fenônemo que faz com o modelo não generalize bem em outros dataset além do usando durante o treino.

# *Overfitting*

Redes neurais são muito flexíveis porque não se limitam a ver cada atributo a ser aprendido individualmente. Em vez disso, elas podem aprender interações entre os atributos. Por causa disso, mesmo quando temos apenas um pequeno número de atributos, as redes neurais profundas são capazes de chegar ao *overfitting*, um cenário onde o modelo aprende a classificar muito bem (as vezes, perfeitamente) as instâncias de treino, porém não generaliza para outras instâncias não vista (como são os casos de amostras do conjunto de validação ou teste).

Para evitar esse cenário, algumas técnicas foram propostas para evitar o *overfitting*.

**Como introduzido, nesta aula prática, implementaremos e testaremos duas técnicas para evitar o *overfitting*: *Dropout* e *Weight Decay***

**ATENÇÃO: a alteração deste bloco pode implicar em problemas na execução dos blocos restantes!**

In [1]:
import torch
import torch.nn.functional as F
import torchvision

from torchvision import datasets, transforms
from torch import optim, nn

import numpy as np

import os
import sys
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids = list(range(n))
device

device(type='cuda')

In [2]:
## carregando dados

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [transforms.Resize(resize)]
    transformer += [transforms.ToTensor()]
    transformer = transforms.Compose(transformer)

    mnist_train = datasets.MNIST(root=root, train=True,download=True, transform=transformer)
    mnist_test = datasets.MNIST(root=root, train=False,download=True, transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# código para carregar o dataset do Fashion-MNIST
# https://github.com/zalandoresearch/fashion-mnist
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [transforms.Resize(resize)]
    transformer += [transforms.ToTensor()]
    transformer = transforms.Compose(transformer)

    mnist_train = datasets.FashionMNIST(root=root, train=True, download=True, transform=transformer)
    mnist_test = datasets.FashionMNIST(root=root, train=False, download=True, transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

In [3]:
# funções básicas
def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

# Função para inicializar pesos da rede
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 0.01) # valores iniciais são uma normal
        m.bias.data.fill_(0)

## *Dropout*

*Dropout* é uma das formas mais interessantes de regularizar sua rede neural. 
A ideia do *Droupout* é simples: durante o passo de *Forward*, alguns neurônios são aleatoriamente "desligados", ou seja, são zerados, e não são utilizados em nenhum processamento.
Em cada passo do *Forward*, neurônios diferentes são "desligados" aleatoriamente, de acordo com uma probabilide pré-definida.
Lembrem-se que esse processo só acontece durante o treino.
Durante o teste, *Dropout* não tem nenhuma ação e todos os neurônios são usados para gerar o resultado fina.

Formalmente, suponha um neurônio com ativação $h$ e um *Dropout* com probabilide $p$ (de zerar ou "desligar" o neurônio).
Logo, essa técnica irá "desligar" a ativação desse neurônio com probabilidade $p$ ou reescala-la baseado na probabilidade de essa unidade de processamento permanecer ativa (isto é, $1-p$):

$$
\begin{aligned}
h' =
\begin{cases}
    0 & \text{ com probabilidade } p \\
    \frac{h}{1-p} & \text{ caso contrário}
\end{cases}
\end{aligned}
$$

Tal método é interessante e chamou a atenção do mundo acadêmico por ser muito simples de implementar e poder impulsar significativamente o desempenho do modelo.

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), para utilizar os benefícios do Dropout basta adicionar a camada homônima (passando como argumento a probabilidade de desligamento dos neurônios) durante a construção da arquitetura.

**Um exemplo é mostrado abaixo utilizando o framework PyTorch.**

Durante o treino, a camada *Dropout* irá "desligar" aleatoriamente algumas saídas da camada anterior (ou equivalentemente, as entradas para a camada subsequente) de acordo com a probabilidade especificada.

Quando o PyTorch não está no modo de treinamento, a camada *Dropout* simplesmente passa os dados sem fazer nenhum "desligamento".

In [5]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 20, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),               # camada densamente conectada
        nn.Dropout(0.2),                   # dropout com 20% de probabilidade de desligar os neurônios
        nn.ReLU(),
        nn.Linear(256, 256),               # camada densamente conectada
        nn.Dropout(0.5),                   # dropout com 50% de probabilidade de desligar os neurônios
        nn.ReLU(),
        nn.Linear(256, 10)                 # camada densamente conectada para classificação
)                     

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr)

# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

training on cuda
epoch 1, train loss 1.1077, train acc 0.568, test loss 0.7163, test acc 0.739, time 5.5 sec
epoch 2, train loss 0.5749, train acc 0.788, test loss 0.5168, test acc 0.815, time 5.8 sec
epoch 3, train loss 0.4837, train acc 0.825, test loss 0.6097, test acc 0.767, time 5.7 sec
epoch 4, train loss 0.4442, train acc 0.838, test loss 0.7559, test acc 0.702, time 5.6 sec
epoch 5, train loss 0.4104, train acc 0.851, test loss 0.4663, test acc 0.832, time 5.4 sec
epoch 6, train loss 0.3891, train acc 0.857, test loss 0.4083, test acc 0.846, time 5.5 sec
epoch 7, train loss 0.3748, train acc 0.863, test loss 0.4335, test acc 0.844, time 6.2 sec
epoch 8, train loss 0.3622, train acc 0.867, test loss 0.4706, test acc 0.826, time 5.7 sec
epoch 9, train loss 0.3504, train acc 0.871, test loss 0.6285, test acc 0.798, time 5.6 sec
epoch 10, train loss 0.3415, train acc 0.874, test loss 0.4274, test acc 0.845, time 5.8 sec
epoch 11, train loss 0.3319, train acc 0.878, test loss 0.3908

## *Weight Decay*

*Weight Decay* (comumente chamado regularização *L2*), é uma das técnicas mais utilizadas para regularizar modelos paramétricos de aprendizado de máquina.
A intuição básica por trás do *Weight Decay* é a noção de que entre todas as funções $f$, a função $f=0$ é a mais simples. Intuitivamente, podemos medir funções pela sua proximidade a zero. Mas quão devemos medir a distância entre uma função e zero? Não há uma resposta correta. De fato, ramos inteiros da matemática são dedicados a responder a esta questão.

Para nossos propósitos atuais, uma interpretação muito simples será suficiente: vamos considerar uma função linear $f(\mathbf{x}) = \mathbf{w}^\top \mathbf{x}$ é simples se o seu vetor de peso $\mathbf{w}$ for pequeno. Podemos medir isso via norma $||\mathbf{w}||^2$. Uma maneira de manter o vetor de peso pequeno é adicionar sua norma como um termo de penalidade ao problema de minimizar a função de perda (ou *loss*). Assim, nós substituímos nosso objetivo original, *minimizar o erro de previsão nos rótulos de treinamento*, com novo objetivo, *minimizar o erro de previsão e o termo de penalidade*. Agora, se o vetor de peso se tornar muito grande, nosso algoritmo de aprendizagem vai encontrar mais lucro minimizando a norma $||\mathbf{w}||^2$ do que minimizando o erro de treinamento. 

Tecnicamente, para uma função de custo qualquer $\mathcal{L}$, a adição do novo termo de penalidade (ou *weight decay*) acontece da seguinte forma:

$$\mathcal{L}(\mathbf{w}, b) + \frac{\lambda}{2} \|\boldsymbol{w}\|^2$$

Esse parâmetro não negativo $\lambda \geq 0$ dita a quantidade de regularização. Para $\lambda = 0$, recuperamos nossa função de perda original, enquanto para $\lambda > 0 $ garantimos que os pesos $\mathbf{w}$ não crescerão demais.

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), *Weight Decay* pode ser facilmente agregado à função de custo durante a construção do modelo.

**Um exemplo é mostrado abaixo utilizando o framework PyTorch.**

In [7]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e valor de weight decay
num_epochs, lr, batch_size, weight_decay = 20, 0.5, 256, 0.005

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),               # camada densamente conectada
        nn.ReLU(),
        nn.Linear(256, 256),               # camada densamente conectada
        nn.ReLU(),
        nn.Linear(256, 10)                 # camada densamente conectada para classificação
)                     

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=weight_decay)


# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

training on cuda
epoch 1, train loss 1.2832, train acc 0.501, test loss 0.8203, test acc 0.692, time 5.6 sec
epoch 2, train loss 0.6402, train acc 0.759, test loss 0.9408, test acc 0.633, time 5.6 sec
epoch 3, train loss 0.6671, train acc 0.756, test loss 0.5815, test acc 0.792, time 5.7 sec
epoch 4, train loss 0.5705, train acc 0.794, test loss 0.7470, test acc 0.712, time 5.6 sec
epoch 5, train loss 0.5616, train acc 0.797, test loss 0.7229, test acc 0.734, time 5.7 sec
epoch 6, train loss 0.5845, train acc 0.787, test loss 0.5289, test acc 0.817, time 5.5 sec
epoch 7, train loss 0.8481, train acc 0.774, test loss 1.8410, test acc 0.325, time 5.6 sec
epoch 8, train loss 0.8498, train acc 0.684, test loss 0.5942, test acc 0.784, time 5.7 sec
epoch 9, train loss 0.5924, train acc 0.783, test loss 0.5341, test acc 0.782, time 6.2 sec
epoch 10, train loss 0.6041, train acc 0.782, test loss 0.7973, test acc 0.706, time 5.3 sec
epoch 11, train loss 0.6649, train acc 0.759, test loss 0.6505

## MNIST - *Overfitting* and regularizadores

Agora usaremos um dataset específico juntamente com um rede mais produnda para tentar mostrar o efeito de *overfitting* e entender como as técnicas de regularização aprendidas podem ser usadas para resolver esse problema.

O modelo implementado abaixo usa o dataset MNIST treinando uma rede com quatro camadas que não usa nenhum método de regularização.
Note a diferença entre o *loss* e a acurácia de treino e teste.
Um resultado onde o *loss* do teste é relativamente maior que o treino (caso do exemplo abaixo) indica que um (neste caso, princípio de) *overfitting* está acontecendo.
Use as técnicas vistas nesta aula prática para tratar esse problema.

Especificamente, implemente:

1. Uma versão dessa arquitetura com *Dropout*. Teste diferentes valores de probabilidade de forma a diminuir o *overfitting*.
2. Uma versão desse modelo com *Weight Decay*. Teste diferentes valores de $\lambda$ de forma a diminuir o *overfitting*.
3. Uma versão que combina os dois métodos de regularização aprendidos. Talvez seja necessário testar diferentes valores para a probabilidade do *Dropout* e do $\lambda$.

In [ ]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 20, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),              
        nn.ReLU(),
        nn.Linear(256, 128),              
        nn.ReLU(),
        nn.Linear(128, 64),              
        nn.ReLU(),
        nn.Linear(64, 10)
)  

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr)

# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20835856.59it/s]                            


Extracting /root/.pytorch/datasets/fashion-mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/MNIST/raw


32768it [00:00, 332417.36it/s]
0it [00:00, ?it/s]

Extracting /root/.pytorch/datasets/fashion-mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/MNIST/raw


1654784it [00:00, 4986039.82it/s]                           
8192it [00:00, 130660.79it/s]


Extracting /root/.pytorch/datasets/fashion-mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/MNIST/raw
Extracting /root/.pytorch/datasets/fashion-mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/MNIST/raw
Processing...
Done!
training on cuda
epoch 1, train loss 2.3017, train acc 0.112, test loss 2.3004, test acc 0.114, time 7.4 sec
epoch 2, train loss 1.8158, train acc 0.286, test loss 1.4500, test acc 0.420, time 7.4 sec
epoch 3, train loss 0.6222, train acc 0.790, test loss 0.3616, test acc 0.892, time 7.4 sec
epoch 4, train loss 0.2093, train acc 0.943, test loss 0.1459, test acc 0.959, time 7.4 sec
epoch 5, train loss 0.1273, train acc 0.964, test loss 0.1348, test acc 0.963, time 7.4 sec
epoch 6, train loss 0.0965, train acc 0.972, test loss 0.1890, test acc 0.947, time 7.5 sec
epoch 7, train loss 0.0746, train acc 0.978, test loss 0.1013, test acc 0.973, time 7.4 sec
epoch 8, train loss 0.0545, train acc 0.984, te

## Exercícios

1. Qual impacto de alterar a probabilidade da camada de *Dropout*?  Como fica uma rede com probabilidade de 50% de "desligar" os neurônios?
1. E em relação ao valor de $\lambda$ para o *Weight Decay*, qual impacto? Teste valores como 0.0005 e 0.0001.
1. Qual efeito da quantidade de *epochs* na acurácia final do modelo?


